## Load DataSet

In [ ]:
from datasets import load_dataset
data = load_dataset("rotten_tomatoes", token="" )

Using the latest cached version of the dataset since rotten_tomatoes couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/seono/.cache/huggingface/datasets/rotten_tomatoes/default/0.0.0/aa13bc287fa6fcab6daf52f0dfb9994269ffea28 (last modified on Sat May 24 19:50:05 2025).


In [4]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

## Create Pipeline

In [ ]:
from transformers import pipeline

model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"

pipe = pipeline(
    model=model_path,
    tokenizer=model_path,
    return_all_scores=True,
    device="cuda:0",
    token=""
)
print("model::\n",pipe.model)
print("Tokenizer::\n",pipe.tokenizer)
print("Model Config::\n",pipe.model.config)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


model::
 RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)


/home/seono/anaconda3/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


## Get Inference

In [31]:
import numpy as np
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

# Run inference
y_pred = []
i =0
Classes = ["negative", "neutral", "positive"]

for output in tqdm(pipe(KeyDataset(data["test"], "text")), total=len(data["test"])):

    labels = np.argmax([output[0]["score"], output[1]["score"], output[2]["score"]])
    score = output[labels]["score"]
    if i < 10:
        print(f"DATA: {data["test"]["text"][i]} LABEL: {Classes[labels]} SCORE: {score}")
    i += 1

    assignment = np.argmax([output[0]["score"], output[2]["score"]])
    y_pred.append(assignment)

  1%|          | 9/1066 [00:00<00:27, 38.33it/s]

DATA: lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness . LABEL: positive SCORE: 0.9546051621437073
DATA: consistently clever and suspenseful . LABEL: positive SCORE: 0.8883835673332214
DATA: it's like a " big chill " reunion of the baader-meinhof gang , only these guys are more harmless pranksters than political activists . LABEL: negative SCORE: 0.7359189987182617
DATA: the story gives ample opportunity for large-scale action and suspense , which director shekhar kapur supplies with tremendous skill . LABEL: positive SCORE: 0.9342906475067139
DATA: red dragon " never cuts corners . LABEL: neutral SCORE: 0.7006850838661194
DATA: fresnadillo has something serious to say about the ways in which extravagant chance can distort our perspective and throw us off the path of good sense . LABEL: negative SCORE: 0.608919620513916
DATA: throws in enough clever and unexpected twists to make the formula feel fresh . L

  2%|▏         | 20/1066 [00:00<00:23, 44.87it/s]

DATA: generates an enormous feeling of empathy for its characters . LABEL: positive SCORE: 0.5940332412719727


100%|██████████| 1066/1066 [00:19<00:00, 54.53it/s]


## Evaluate Performance

In [29]:
from sklearn.metrics import classification_report

def evaluate_performance(y_true, y_pred):
    """Create and print the classification report"""
    performance = classification_report(
        y_true, y_pred,
        target_names=["Negative Review", "Positive Review"]
    )
    print(performance)

evaluate_performance(data["test"]["label"], y_pred)

                 precision    recall  f1-score   support

Negative Review       0.76      0.88      0.81       533
Positive Review       0.86      0.72      0.78       533

       accuracy                           0.80      1066
      macro avg       0.81      0.80      0.80      1066
   weighted avg       0.81      0.80      0.80      1066

